In [3]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from time import time
from sys import argv

In [4]:
def _time(f):
    def wrapper(*args):
        start = time()
        r = f(*args)
        duration = time() - start
        print(f"{f.__name__} : {duration}")
        return r
    return wrapper


In [5]:
@_time
def for_loop_split(image: np.ndarray, kernel_size: tuple):
    h,w,c = image.shape
    t_h, t_w = kernel_size

    tile = np.zeros((h//t_h, w//t_w, t_h, t_w, c))
    y = x = 0
    for i in range(0, h, t_h):
        for j in range(0, w, t_w):
            tile[y][x] = image[i:i+t_h, j:j+t_w, :c]
            x += 1
        y += 1
        x = 0
    return tile


In [6]:
@_time
def stride_split(image: np.ndarray, kernel_size: tuple):
    h, w, c = image.shape
    t_h, t_w = kernel_size

    bytelength = image.nbytes // image.size
    tile = np.lib.stride_tricks.as_strided(image,
                                           shape=(h//t_h, w//t_w, t_h, t_w, c),
                                           strides=(w*t_h*bytelength*c, t_w*bytelength*c, w*bytelength*c, bytelength*c, bytelength))
    return tile

In [7]:
@_time
def reshape_split(image: np.ndarray, kernel_size: tuple):
    h, w, c = image.shape
    t_h, t_w = kernel_size

    tile = image.reshape(h//t_h, t_h, w//t_w, t_w, c)
    tile = tile.swapaxes(1, 2)
    return tile

In [9]:
img = np.asarray(Image.open("/home/z/MVPC10/CODE/pplcnt_model/in/test/acqui2_image_1697_png.rf.783a5471c9661f67563da5a2b1594272.jpg"))

t1, t2 = 10, 10
tilesize = (int(t1), int(t2))

tile1 = for_loop_split(img, tilesize)
tile2 = stride_split(img, tilesize)
tile3 = reshape_split(img, tilesize)

if (tile1 == tile2).all() and (tile2 == tile3).all():
    n = tile1.shape[0]*tile1[1]
    print(f"all tiles are same : {n}")

for_loop_split : 0.00015687942504882812
stride_split : 3.2901763916015625e-05
reshape_split : 8.821487426757812e-06
all tiles are same : [[[[ 304.  304.  304.]
   [ 144.  144.  144.]
   [ 224.  224.  224.]
   ...
   [   0.    0.    0.]
   [   0.    0.    0.]
   [ 248.  248.  248.]]

  [[ 112.  112.  112.]
   [   0.    0.    0.]
   [ 224.  224.  224.]
   ...
   [ 120.  120.  120.]
   [  24.   24.   24.]
   [   0.    0.    0.]]

  [[ 144.  144.  144.]
   [  16.   16.   16.]
   [ 264.  264.  264.]
   ...
   [   0.    0.    0.]
   [  40.   40.   40.]
   [ 112.  112.  112.]]

  ...

  [[   0.    0.    0.]
   [   0.    0.    0.]
   [ 120.  120.  120.]
   ...
   [ 104.  104.  104.]
   [  72.   72.   72.]
   [   0.    0.    0.]]

  [[  56.   56.   56.]
   [  80.   80.   80.]
   [   0.    0.    0.]
   ...
   [   0.    0.    0.]
   [  56.   56.   56.]
   [   0.    0.    0.]]

  [[   0.    0.    0.]
   [  24.   24.   24.]
   [2040. 2040. 2040.]
   ...
   [  72.   72.   72.]
   [  48.   48.   48.]